### 이 코드는... 
 - 스터디에서 Generating한 학습용 처방전 이미지 파일과, 라벨링 좌표 및 라벨을 포함한 json 파일을 YOLO 데이터셋 형식으로 변환하는 코드.
 - 먼저 YOLO 데이터셋에 맞도록 폴더를 생성한다. 
 - Generating한 이미지 데이터를 불러와 생성한 데이터셋 폴더에 복사한다. 
 - Generating한 Json 파일의 좌표 (x1, y1, x2, y2)를 YOLO가 학습할 수 있는 데이터로 transform해주는 pybboxes 라이브러리를 활용하여 txt파일을 생성한다. 
 - 복사할때 학습데이터, 테스트데이터, 검증데이터를 7:2:1로 나누어 복사한다. 
 - 복사할때 이미지파일과 만들어질 텍스트파일이 1:1로 매칭되도록 복사한다. 
 - YOLOv5가 학습데이터를 읽어들일 수 있도록 환경설정 파일인 data.yaml을 생성한다. 


In [ ]:
# def create_yolo_folder_structure(base_dir):
#     os.makedirs(os.path.join(base_dir, 'YOLO', 'test', 'images'), exist_ok=True)
#     os.makedirs(os.path.join(base_dir, 'YOLO', 'test', 'labels'), exist_ok=True)
#     os.makedirs(os.path.join(base_dir, 'YOLO', 'train', 'images'), exist_ok=True)
#     os.makedirs(os.path.join(base_dir, 'YOLO', 'train', 'labels'), exist_ok=True)
#     os.makedirs(os.path.join(base_dir, 'YOLO', 'valid', 'images'), exist_ok=True)
#     os.makedirs(os.path.join(base_dir, 'YOLO', 'valid', 'labels'), exist_ok=True)
    
# if __name__ == "__main__":
#     # 원하는 기본 경로 설정
#     base_directory = r"..\..\1.data\4.dataSet"
#     create_yolo_folder_structure(base_directory)

In [ ]:
# # 재영 진행(라벨을 비율대로)
# W, H = 595, 842

# # json 파일들이 저장되어 있는 디렉토리 경로
# json_dir_path = r"..\..\1.data\4.dataSet\json"

# # json 디렉토리 내의 모든 파일 이름 가져오기
# json_files = os.listdir(json_dir_path)

# # 모든 json 파일에 대해 반복
# for json_file_name in json_files:

#     # 파일 확장자가 .json인 경우에만 처리
#     if json_file_name.endswith(".json"):
#         json_path = os.path.join(json_dir_path, json_file_name)

#         with open(json_path, 'r', encoding='utf-8') as json_file:
#             data = json.load(json_file)

#         points = []
#         for data_ in data:
#             point = [0]
#             x1 = int(data_['x1'])
#             y1 = int(data_['y1'])
#             x2 = int(data_['x2'])
#             y2 = int(data_['y2'])
#             voc_bbox = (x1, y1, x2, y2)
#             temp = list(pbx.convert_bbox(voc_bbox, from_type="voc", to_type="yolo", image_size=(W,H)))
#             point += temp
#             points.append(point)
        
#         tmp = ""
#         for id, x, y, w, h in points:
#             tmp += f'{id} {x} {y} {w} {h}\n'

#         # txt 파일 이름을 json 파일 이름에 맞춰서 변경
#         txt_file_name = './txt/' + json_file_name.replace('.json', '.txt')

#         with open(txt_file_name, 'w+') as txt_file:
#             txt_file.write(tmp)

# src_paths = './txt'
# def split_files_to_folders(n):
    
#     src_path = './txt'
#     train_dir = '../../1.data/4.dataSet/YOLO/train/labels'
#     test_dir = '../../1.data/4.dataSet/YOLO/test/labels'
#     valid_dir = '../../1.data/4.dataSet/YOLO/valid/labels'

#     os.makedirs(train_dir, exist_ok=True)
#     os.makedirs(test_dir, exist_ok=True)
#     os.makedirs(valid_dir, exist_ok=True)

#     files_list = os.listdir(src_path)

#     #random.shuffle(files_list)

#     total_files = len(files_list)
#     train_files = int(total_files * 0.7)
#     test_files = int(total_files * 0.2)

#     train_files_list = files_list[:train_files]
#     test_files_list = files_list[train_files:train_files + test_files]
#     valid_files_list = files_list[train_files + test_files:]

#     for file_name in train_files_list:
#         shutil.copy(os.path.join(src_path, file_name), os.path.join(train_dir, file_name))

#     for file_name in test_files_list:
#         shutil.copy(os.path.join(src_path, file_name), os.path.join(test_dir, file_name))

#     for file_name in valid_files_list:
#         shutil.copy(os.path.join(src_path, file_name), os.path.join(valid_dir, file_name))

# if __name__ == '__main__':
#     n = len(os.listdir(src_paths))
#     split_files_to_folders(n)




In [6]:
#!pip install pybboxes
import os
import json
import shutil
import numpy as np
import pybboxes as pbx
import yaml

In [7]:
# 보경님 코드 

def create_yolo_folder_structure(base_dir):
    os.makedirs(os.path.join(base_dir, 'YOLO', 'test', 'images'), exist_ok=True)
    os.makedirs(os.path.join(base_dir, 'YOLO', 'test', 'labels'), exist_ok=True)
    os.makedirs(os.path.join(base_dir, 'YOLO', 'train', 'images'), exist_ok=True)
    os.makedirs(os.path.join(base_dir, 'YOLO', 'train', 'labels'), exist_ok=True)
    os.makedirs(os.path.join(base_dir, 'YOLO', 'valid', 'images'), exist_ok=True)
    os.makedirs(os.path.join(base_dir, 'YOLO', 'valid', 'labels'), exist_ok=True)

base_directory = r"..\..\1.data\4.dataSet"
create_yolo_folder_structure(base_directory)

source_img_dir = r"..\..\1.data\4.dataSet\img"
source_json_dir = r"..\..\1.data\4.dataSet\json"

img_files = os.listdir(source_img_dir)
json_files = os.listdir(source_json_dir)


pairs = list(zip(sorted(img_files), sorted(json_files)))
np.random.shuffle(pairs)

n = len(pairs)
n_test = int(n * 0.2)
n_train = int(n * 0.7)
n_valid = n - n_test - n_train

W, H = 595, 842

for i, (img_file_name, json_file_name) in enumerate(pairs):
    if i < n_test:
        img_dest_dir = r"..\..\1.data\4.dataSet\YOLO\test\images"
        json_dest_dir = r"..\..\1.data\4.dataSet\YOLO\test\labels"
    elif i < n_test + n_train:
        img_dest_dir = r"..\..\1.data\4.dataSet\YOLO\train\images"
        json_dest_dir = r"..\..\1.data\4.dataSet\YOLO\train\labels"
    else:
        img_dest_dir = r"..\..\1.data\4.dataSet\YOLO\valid\images"
        json_dest_dir = r"..\..\1.data\4.dataSet\YOLO\valid\labels"

    # move image file
    source_img_path = os.path.join(source_img_dir, img_file_name)
    destination_img_path = os.path.join(img_dest_dir, img_file_name)
    shutil.copy(source_img_path, destination_img_path)

    # process json file and save as txt
    json_path = os.path.join(source_json_dir, json_file_name)
    with open(json_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    points = []
    for data_ in data:
        point = [0]
        x1 = int(data_['x1'])
        y1 = int(data_['y1'])
        x2 = int(data_['x2'])
        y2 = int(data_['y2'])
        voc_bbox = (x1, y1, x2, y2)
        temp = list(pbx.convert_bbox(voc_bbox, from_type="voc", to_type="yolo", image_size=(W,H)))
        point += temp
        points.append(point)

    tmp = ""
    for id, x, y, w, h in points:
        tmp += f'{id} {x} {y} {w} {h}\n'

    txt_file_name = json_file_name.replace('.json', '.txt')
    destination_txt_path = os.path.join(json_dest_dir, txt_file_name)
    with open(destination_txt_path, 'w+') as txt_file:
        txt_file.write(tmp)


In [8]:
# yaml파일에 들어갈 라벨과 클래스를 설정합니다.
num_classess = 1
class_names = ["text"]

# yaml파일을 생성하기 위한 경로 정의(path, train, test, val)
yolo_dataset_path = "../../1.data/4.dataSet/YOLO"
yolo_train_path = "../../1.data/4.dataSet/YOLO/train/images"
yolo_test_path = "../../1.data/4.dataSet/YOLO/test/images"
yolo_val_path = "../../1.data/4.dataSet/YOLO/valid/images"


data_dict = {
    "path" : yolo_dataset_path,
    "train" : yolo_train_path, 
    "test" : yolo_test_path,
    "val" : yolo_val_path,
    "nc" : num_classess,
    "names" : class_names
}

with open("../../1.data/4.dataSet/YOLO/data.yaml", "w", encoding="utf-8") as f:
    yaml.dump(data_dict, f, default_flow_style=False, allow_unicode=True)

In [19]:
# 이건 안돌아감. bash에서 명령어로 직접 실행

from yolov5.train import train

# 학습 환경 설정
yaml_config = r"C:\Users\excel\miniconda3\envs\yeardream\Lib\site-packages\yolov5\models\yolov5s.yaml"  # YOLOv5 아키텍처 설정 파일 경로
data_yaml = "../../1.data/4.dataSet/YOLO/data.yaml"  # 데이터셋 정보가 담긴 data.yaml 파일 경로
# weights = "yolov5s.pt"  # 사전 학습된 모델의 가중치 (필요한 경우)
epochs = 100  # 훈련 에폭 수 변경

class Options:
    def __init__(self, epochs, data, cfg, resume, name, save_dir, batch_size=16, weights=None, single_cls=False, evolve=False, noval=False, nosave=False, workers=0, freeze=False):
        self.epochs = epochs
        self.data = data
        self.cfg = cfg
        self.resume = resume
        self.name = name
        self.save_dir = save_dir
        self.batch_size = batch_size
        self.weights = weights
        self.single_cls = single_cls
        self.evolve = evolve
        self.noval = noval
        self.nosave = nosave
        self.workers = workers
        self.freeze = freeze

opt = Options(epochs=epochs, data=data_yaml, cfg=yaml_config, resume=False, name="", save_dir='../0.test_code/')

train(hyp=)


AttributeError: module 'yolov5.train' has no attribute 'data_yaml'

In [ ]:
# # 보경쓰 코드 
# # 필요한 모듈을 임포트 합니다.
# import json
# import os
# import pybboxes as pbx
# import shutil

# # YOLO 구조 폴더를 만드는 함수입니다.
# def create_yolo_folder_structure(base_dir):
#     os.makedirs(os.path.join(base_dir, 'YOLO', 'test', 'labels'), exist_ok=True)
#     os.makedirs(os.path.join(base_dir, 'YOLO', 'train', 'labels'), exist_ok=True)
#     os.makedirs(os.path.join(base_dir, 'YOLO', 'valid', 'labels'), exist_ok=True)

# # 이미지 파일을 YOLO 구조 폴더로 이동하는 함수입니다.
# def move_files(source_dir, destination_dir):
#     if not os.path.exists(destination_dir):
#         os.makedirs(destination_dir)

#     for file_name in os.listdir(source_dir):
#         source_path = os.path.join(source_dir, file_name)
#         destination_path = os.path.join(destination_dir, file_name)
    
#         if os.path.isfile(source_path):
#             shutil.move(source_path, destination_path)

# # json 파일을 YOLO 형식의 txt 파일로 변환하는 함수입니다.
# def json_to_yolo(json_dir_path, destination_dir, W, H):
#     # json 디렉토리 내의 모든 파일 이름 가져오기
#     json_files = os.listdir(json_dir_path)

#     # 모든 json 파일에 대해 반복
#     for json_file_name in json_files:
#         # 파일 확장자가 .json인 경우에만 처리
#         if json_file_name.endswith(".json"):
#             json_path = os.path.join(json_dir_path, json_file_name)

#             with open(json_path, 'r', encoding='utf-8') as json_file:
#                 data = json.load(json_file)

#             points = []
#             for data_ in data:
#                 point = [0]
#                 x1 = int(data_['x1'])
#                 y1 = int(data_['y1'])
#                 x2 = int(data_['x2'])
#                 y2 = int(data_['y2'])
#                 voc_bbox = (x1, y1, x2, y2)
#                 temp = list(pbx.convert_bbox(voc_bbox, from_type="voc", to_type="yolo", image_size=(W,H)))
#                 point += temp
#                 points.append(point)
            
#             tmp = ""
#             for id, x, y, w, h in points:
#                 tmp += f'{id} {x} {y} {w} {h}\n'

#             # txt 파일 이름을 json 파일 이름에 맞춰서 변경
#             txt_file_name = os.path.join(destination_dir, json_file_name.replace('.json', '.txt'))

#             with open(txt_file_name, 'w+') as txt_file:
#                 txt_file.write(tmp)

# if __name__ == "__main__":
#     # 원하는 기본 경로 설정
#     base_directory = r"..\..\1.data\4.dataSet"

#     # YOLO 구조 폴더 생성
#     create_yolo_folder_structure(base_directory)

#     # 이미지 파일 이동
#     source_dir = r"..\..\1.data\4.dataSet\img"
#     test_img_dir = r"..\..\1.data\4.dataSet\YOLO\test\images"
#     train_img_dir = r"..\..\1.data\4.dataSet\YOLO\train\images"
#     valid_img_dir = r"..\..\1.data\4.dataSet\YOLO\valid\images"
#     move_files(source_dir, test_img_dir)
#     move_files(source_dir, train_img_dir)
#     move_files(source_dir, valid_img_dir)

#     # json 파일을 YOLO 형식의 txt 파일로 변환
#     json_dir_path = r"..\..\1.data\4.dataSet\json"
#     W, H = 595, 842
#     test_label_dir = r"..\..\1.data\4.dataSet\YOLO\test\labels"
#     train_label_dir = r"..\..\1.data\4.dataSet\YOLO\train\labels"
#     valid_label_dir = r"..\..\1.data\4.dataSet\YOLO\valid\labels"
#     json_to_yolo(json_dir_path, test_label_dir, W, H)
#     json_to_yolo(json_dir_path, train_label_dir, W, H)
#     json_to_yolo(json_dir_path, valid_label_dir, W, H)

In [ ]:
import json

# JSON 파일 열기


# # 데이터를 문자열로 변환
# data_str = json.dumps(data, indent=4)

# # 텍스트 파일로 저장
# with open('output.txt', 'w') as text_file:
#     text_file.write(data_str)

In [ ]:

# def test(json):
#     width=x2-x1
#     height=y2-y1
#     center_x=(x2+x1)/2
#     center_y=(y2+y1)/2
#     return width,height,center_x,center_y